# Bloco transformer

**Descrição:**
Implementação do *Transformer Block* (estilo GPT) — o principal “tijolo” arquitetural de modelos *decoder-only*.  
Ele combina **atenção multi-cabeças com máscara causal**, **rede feed-forward (MLP)**, **normalização** e **conexões residuais**, preservando o formato do tensor ao longo do bloco (mesmo `batch`, `seq_len` e `emb_dim`).

**Objetivo:**
Receber uma sequência de embeddings e **produzir embeddings contextualizados**, isto é, representações em que cada token passa a carregar informação relevante dos tokens anteriores (dependências de longo alcance), ao mesmo tempo em que refina a informação local via MLP. 

**Funcionamento:**
1. **Pré-normalização (Pre-LN) + Atenção Causal Multi-Head**  
   - Aplica *LayerNorm* na entrada do bloco.  
   - Calcula `Q, K, V`, aplica **máscara causal** (impede “olhar o futuro”), obtém pesos de atenção e combina os valores.  
   - Aplica *dropout* e projeta a saída.
2. **Atalho residual (skip connection)**  
   - Soma a saída da atenção com a entrada original do bloco.
3. **Pré-normalização (Pre-LN) + Feed-Forward (MLP)**  
   - Aplica *LayerNorm* novamente.  
   - Passa por um MLP (tipicamente `Linear -> GELU -> Linear`) para transformar cada posição de forma independente.
4. **Atalho residual final**  
   - Soma a saída do MLP com o tensor após o primeiro residual.  

Resultado: um bloco que **mantém a mesma dimensionalidade de entrada e saída**, mas “re-encoda” cada token com contexto + não-linearidades, pronto para ser empilhado várias vezes numa arquitetura GPT. 

![Bloco transformer](../../imagens/cap04/04_bloco_transformer.png)

*Baseado na figura do livro, Capítulo 4, página 114*

In [1]:
from typing import Any

import torch
from torch import nn

from build_llm.activation import GELU
from build_llm.attention import MultiHeadAttention
from build_llm.layer import LayerNorm

### Verificando a presença da GPU

In [2]:
print(torch.__version__)  # Versão do torch
print(torch.cuda.is_available())  # Verificação de GPU
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando:", device)

2.9.1+cpu
False
Usando: cpu


## Configuração do modelo

As mesmas utilizadas no notebook [01 - LLM architecture](./01%20-%20LLM%20architecture.ipynb)

In [3]:
GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50257,  # Tamanho do vocabulário
    "context_length": 1024,  # Comprimento do contexto
    "emb_dim": 768,  # Dimensão do embedding
    "n_heads": 12,  # Número de cabeças de atenção
    "n_layers": 12,  # Número de camadas
    "drop_rate": 0.1,  # Taxa de dropout
    "qkv_bias": False,  # Viés em Query-Key-Value (QKV)
}

## Classe `FeedForward` (MLP do Transformer)

A classe **`FeedForward`** implementa a **rede feed-forward** (também chamada de **MLP** ou *position-wise feed-forward network*) que aparece dentro de cada **bloco Transformer** (no estilo GPT).

Ela é composta por **duas camadas lineares** com uma **ativação não-linear (GELU)** no meio:

- **Expansão**: `emb_dim -> 4 * emb_dim`  
- **Não-linearidade**: `GELU()`  
- **Projeção de volta**: `4 * emb_dim -> emb_dim`

### Para que ela serve?

Dentro do Transformer, existem dois “tipos” principais de processamento:

1. **Self-Attention (atenção)**: permite que cada token “olhe” para outros tokens e misture informações do contexto.
2. **FeedForward / MLP (esta classe)**: aplica uma transformação **não-linear** em **cada token individualmente**, de forma independente (posição a posição).

Ou seja, depois que a atenção combinou informações entre tokens, o `FeedForward` atua como um “processador” por token, ajudando o modelo a:

- aumentar a **capacidade de representação** (aprendendo combinações mais complexas),
- introduzir **não-linearidades** (o que torna o modelo muito mais expressivo),
- refinar e re-encodar os vetores de embedding mantendo o mesmo tamanho final (`emb_dim`).

### Por que usar `4 * emb_dim`?

O fator `4` é uma convenção comum em arquiteturas do tipo GPT/Transformer:

- ao **expandir** a dimensão interna, o modelo ganha um “espaço” maior para computações intermediárias;
- ao **reduzir** de volta para `emb_dim`, o bloco mantém compatibilidade com o residual e com o restante da rede.

### Entrada e saída

- **Entrada**: tensor com shape `(..., emb_dim)` (por exemplo, `[batch, tokens, emb_dim]`)
- **Saída**: tensor com o **mesmo shape** da entrada `(..., emb_dim)`

Isso é importante porque o `FeedForward` normalmente é usado em conjunto com **conexões residuais** no `TransformerBlock`.


In [4]:
def get_device() -> torch.device:
    """
    Retorna o device apropriado (CUDA se disponível, caso contrário CPU).
    """
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class FeedForward(nn.Module):
    """
    Rede feed-forward (MLP) usada dentro de um bloco Transformer (estilo GPT).

    Estrutura:
    ----------
    Linear(emb_dim -> 4*emb_dim) -> GELU -> Linear(4*emb_dim -> emb_dim)

    Parâmetros:
    ----------
    cfg : dict
        Dicionário de configuração contendo:
        - "emb_dim" (int): dimensão do embedding / hidden size do modelo.

    Exceções:
    --------
    Levanta KeyError se "emb_dim" não existir em cfg.
    Levanta TypeError / ValueError se "emb_dim" não puder ser convertido para int
    ou não for um inteiro positivo.
    """

    def __init__(self, cfg: dict) -> None:
        super().__init__()

        if "emb_dim" not in cfg:
            raise KeyError('cfg deve conter a chave "emb_dim".')

        try:
            emb_dim = int(cfg["emb_dim"])
        except (TypeError, ValueError) as e:
            raise TypeError(
                '"emb_dim" deve ser um inteiro (ou conversível para int).'
            ) from e

        if emb_dim <= 0:
            raise ValueError('"emb_dim" deve ser um inteiro positivo.')

        self.layers = nn.Sequential(
            nn.Linear(emb_dim, 4 * emb_dim),
            GELU(),
            nn.Linear(4 * emb_dim, emb_dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Executa o forward pass do feed-forward.

        Parâmetros:
        ----------
        x : torch.Tensor
            Tensor de entrada com shape (..., emb_dim).

        Retorno:
        -------
        torch.Tensor
            Tensor de saída com o mesmo shape de entrada (..., emb_dim).
        """
        return self.layers(x)

## Classe `TransformerBlock` (Bloco principal do Transformer estilo GPT)

A classe **`TransformerBlock`** implementa um **bloco Transformer completo**, que é a unidade básica repetida várias vezes para formar um modelo do tipo GPT.

Ela combina quatro ideias fundamentais:

1. **Multi-Head Self-Attention (MHA)**  
   Permite que cada token “preste atenção” em outros tokens da sequência, misturando informações do contexto.

2. **FeedForward / MLP (FFN)**  
   Processa cada token individualmente com camadas densas e não-linearidade, aumentando a capacidade do modelo.

3. **Pre-LayerNorm (LayerNorm antes de cada sub-bloco)**  
   Normaliza a entrada de cada sub-bloco (atenção e MLP), ajudando a estabilizar o treinamento (muito comum em modelos GPT modernos).

4. **Conexões residuais + Dropout**  
   Usa *skip connections* (atalhos) para facilitar o fluxo de gradientes e aplica dropout para regularização.

### Fluxo interno do bloco

O bloco é composto por **dois sub-blocos**, cada um com:
- normalização,
- operação principal,
- dropout,
- soma residual.

#### 1) Sub-bloco de Atenção
```
shortcut = x
x = LayerNorm(x)
x = MultiHeadAttention(x)
x = Dropout(x)
x = x + shortcut
```

Aqui, a atenção mistura informação entre tokens, e o residual garante que o sinal original possa “passar direto” caso a atenção não precise alterá-lo muito.

#### 2) Sub-bloco FeedForward (MLP)
```
shortcut = x
x = LayerNorm(x)
x = FeedForward(x)
x = Dropout(x)
x = x + shortcut
```

Aqui, o MLP refina a representação **token a token** (posição a posição), adicionando expressividade e não-linearidade.

### Para que serve esse bloco?

O `TransformerBlock` é responsável por transformar embeddings iniciais em representações cada vez mais ricas, combinando:

- **contexto** (via self-attention),
- **capacidade não-linear** (via MLP),
- **estabilidade e eficiência no treinamento** (via LayerNorm + residual).

Empilhando vários `TransformerBlock`s, o modelo vai construindo camadas de entendimento:  
camadas mais baixas capturam padrões locais/sintáticos, enquanto camadas mais altas tendem a capturar relações mais abstratas (dependendo do tamanho/dados do modelo).

### Entrada e saída

- **Entrada**: `x` com shape **`[batch_size, num_tokens, emb_dim]`**
- **Saída**: mesmo shape **`[batch_size, num_tokens, emb_dim]`**

Isso é essencial para:
- empilhar vários blocos em sequência,
- manter compatibilidade com conexões residuais,
- manter a mesma “largura” do modelo ao longo das camadas.

### Papel dos principais parâmetros (`cfg`)

- **`emb_dim`**: tamanho do vetor de embedding (largura do modelo)
- **`context_length`**: limite máximo de tokens (tamanho do contexto)
- **`n_heads`**: número de cabeças de atenção (divide `emb_dim` em partes)
- **`drop_rate`**: intensidade do dropout (regularização)
- **`qkv_bias`**: se a atenção usa bias nas projeções de Q/K/V

> Observação: o código valida que `emb_dim % n_heads == 0` para garantir que cada head tenha a mesma dimensão interna.



In [6]:
class TransformerBlock(nn.Module):
    """
    Bloco Transformer (estilo GPT) com:
    - Multi-Head Self-Attention
    - MLP / FeedForward
    - Pre-LayerNorm (LayerNorm antes de cada sub-bloco)
    - Conexões residuais (shortcut) e dropout nas saídas dos sub-blocos

    Fluxo (simplificado):
    ---------------------
    x -> LN -> MHA -> Dropout -> +residual
      -> LN -> FFN -> Dropout -> +residual

    Parâmetros:
    ----------
    cfg : dict
        Dicionário de configuração contendo:
        - "emb_dim" (int): dimensão do embedding (d_model).
        - "context_length" (int): tamanho máximo de contexto (n_tokens).
        - "n_heads" (int): número de cabeças de atenção.
        - "drop_rate" (float): taxa de dropout (0.0 a 1.0).
        - "qkv_bias" (bool): se deve usar bias em QKV no attention.

    Exceções:
    --------
    Levanta KeyError se alguma chave obrigatória não existir em cfg.
    Levanta TypeError/ValueError se algum valor não puder ser convertido
    para o tipo esperado ou se estiver fora de um intervalo válido.
    """

    def __init__(self, cfg: dict) -> None:
        super().__init__()

        required_keys = [
            "emb_dim",
            "context_length",
            "n_heads",
            "drop_rate",
            "qkv_bias",
        ]
        missing = [k for k in required_keys if k not in cfg]
        if missing:
            raise KeyError(f"cfg está faltando as chaves obrigatórias: {missing}")

        # Validações / conversões defensivas
        try:
            emb_dim = int(cfg["emb_dim"])
            context_length = int(cfg["context_length"])
            n_heads = int(cfg["n_heads"])
            drop_rate = float(cfg["drop_rate"])
            qkv_bias = bool(cfg["qkv_bias"])
        except (TypeError, ValueError) as e:
            raise TypeError(
                "cfg contém valores em formato inválido para o TransformerBlock."
            ) from e

        if emb_dim <= 0:
            raise ValueError('"emb_dim" deve ser um inteiro positivo.')
        if context_length <= 0:
            raise ValueError('"context_length" deve ser um inteiro positivo.')
        if n_heads <= 0:
            raise ValueError('"n_heads" deve ser um inteiro positivo.')
        if emb_dim % n_heads != 0:
            raise ValueError('"emb_dim" deve ser divisível por "n_heads".')
        if not (0.0 <= drop_rate <= 1.0):
            raise ValueError('"drop_rate" deve estar entre 0.0 e 1.0.')

        self.att = MultiHeadAttention(
            d_in=emb_dim,
            d_out=emb_dim,
            context_length=context_length,
            num_heads=n_heads,
            dropout=drop_rate,
            qkv_bias=qkv_bias,
        )

        self.ff = FeedForward(cfg)

        self.norm1 = LayerNorm(emb_dim)
        self.norm2 = LayerNorm(emb_dim)

        # Dropout aplicado na saída de cada sub-bloco antes de somar o residual
        self.drop_shortcut = nn.Dropout(drop_rate)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Executa o forward pass do bloco Transformer.

        Parâmetros:
        ----------
        x : torch.Tensor
            Tensor de entrada com shape [batch_size, num_tokens, emb_dim].

        Retorno:
        -------
        torch.Tensor
            Tensor de saída com o mesmo shape [batch_size, num_tokens, emb_dim].
        """
        # Residual + Attention
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # [batch, tokens, emb_dim]
        x = self.drop_shortcut(x)
        x = x + shortcut

        # Residual + FeedForward
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)  # [batch, tokens, emb_dim]
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

Esse trecho faz um **teste rápido de “forma” (shape check)** do `TransformerBlock`:

* `block = TransformerBlock(GPT_CONFIG_124M)`: cria um bloco Transformer com as configurações do GPT 124M (por exemplo, `emb_dim = 768`, número de heads, dropout, etc.).
* `x = torch.randn(2, 16, 768)`: cria uma entrada aleatória simulando **2 exemplos (batch=2)**, com **16 tokens** cada, e cada token representado por um vetor de **768** dimensões.
* `y = block(x)`: passa `x` pelo bloco (LayerNorm + atenção + residual + LayerNorm + MLP + residual).

O resultado `torch.Size([2, 16, 768]) -> torch.Size([2, 16, 768])` acontece porque o bloco **transforma as representações internamente**, mas **sempre retorna a mesma dimensão `emb_dim`** para poder empilhar vários blocos e manter as conexões residuais.


In [7]:
device = get_device()
print("Usando:", device)

# Envia o bloco para o device (CUDA se existir)
block = TransformerBlock(GPT_CONFIG_124M).to(device)

x = torch.randn(2, 16, GPT_CONFIG_124M["emb_dim"])
y = block(x)
print(x.shape, "->", y.shape)

Usando: cpu
torch.Size([2, 16, 768]) -> torch.Size([2, 16, 768])
